In [7]:
# 1200x1010

DATA_INICIO = '2018-01-01 00:00:00'
DATA_FIM = '2025-12-31 23:59:59'
PALAVRAS_CHAVE = ["Vlr", "Valor", "vlr", "valor"]

In [8]:
# 1200x1010

import pandas as pd

a = pd.read_parquet('C:\\AFL_GABRIEL\\csv_parser\\DATA\\sis\\s1200')
b = pd.read_parquet('C:\\AFL_GABRIEL\\csv_parser\\DATA\\sis\\s1010')

print(len(a))
print(len(b))

264332
3338


In [9]:
# 1200x1010

import pandas as pd
import pyperclip
import os

relatorios = ['1200', '1010']

sys_df_1010 = None
sys_df_1200 = None
df_sys = None
parquet_path_sys = None

for relatorio in relatorios:
    relatorio_sis = f's{relatorio}'

    try:
        parquet_path_sys = f'C:\\AFL_GABRIEL\\csv_parser\\DATA\\sis\\{relatorio_sis}\\'

        df_sys = pd.read_parquet(parquet_path_sys, engine='pyarrow')

        df_sys['periodo'] = (
                df_sys['id'].astype(str).str[17:21] + '-' +
                df_sys['id'].astype(str).str[21:23] + '-' +
                df_sys['id'].astype(str).str[23:25] + ' ' +
                df_sys['id'].astype(str).str[25:27] + ':' +
                df_sys['id'].astype(str).str[27:29]
        )

        df_sys['periodo'] = pd.to_datetime(df_sys['periodo'])
        dt_ini = pd.to_datetime(DATA_INICIO)
        dt_fim = pd.to_datetime(DATA_FIM)

        if relatorio != '1010':
            df_sys = df_sys[(df_sys['periodo'] >= dt_ini) & (df_sys['periodo'] <= dt_fim)]

    except Exception as e:
        print(f"Erro ao carregar {parquet_path_sys}: {e}")

    if relatorio == '1010':
        sys_df_1010 = df_sys
    elif relatorio == '1200':
        sys_df_1200 = df_sys

# Renomeia colunas
s1200 = sys_df_1200.copy().reset_index()
s1010 = sys_df_1010.copy()


In [10]:
# 1200x1010

s1200.rename(columns={
            'codigo_rubrica': 'cod_rubrica',
            'identificador_rubrica': 'id_rubrica',
            'periodo': 'periodo',
            'id': 'id'
        }, inplace=True)

s1010.rename(columns={
            'codigo_rubrica': 'cod_rubrica',
            'identificacao_rubrica': 'id_rubrica',
            'periodo': 'periodo_1010',
            'id': 'id_1010'
        }, inplace=True)

s1200['periodo'] = pd.to_datetime(s1200['periodo'])
s1010['periodo_1010'] = pd.to_datetime(s1010['periodo_1010'])

# 1ª Consolidação: período_1010 <= período
merged = s1200.merge(
            s1010,
            on=['cod_rubrica', 'id_rubrica'],
            suffixes=('', '_1010')
        )

valid = merged[merged['periodo_1010'] <= merged['periodo']]
idx = valid.groupby(['id', 'cod_rubrica', 'id_rubrica', 'index'])['periodo_1010'].idxmax()
s12_final = valid.loc[idx].copy()

colunas_base = sys_df_1200.columns.drop('periodo')
colunas_extras = ['index', 'id_1010', 'descricao_rubrica', 'tipo_da_rubrica', 'incidencia_previdencia_social']
s12_final.rename(columns={
    'numero_do_recibo': 'numero_do_recibo',
    'periodo': 'periodo',
    'id_rubrica': 'identificador_rubrica',
    'cod_rubrica': 'codigo_rubrica',
    'id': 'id'
}, inplace=True)
s12_final = s12_final[list(colunas_base) + colunas_extras]

# Filtra não consolidados
s1200_restante = s1200[~s1200['index'].isin(s12_final['index'])]

merged_restante = s1200_restante.merge(
            s1010,
            on=['cod_rubrica', 'id_rubrica'],
            suffixes=('', '_1010'),
            how='left'
        )

merged_restante.rename(columns={
            'numero_do_recibo': 'numero_do_recibo',
            'periodo': 'periodo',
            'id_rubrica': 'identificador_rubrica',
            'cod_rubrica': 'codigo_rubrica',
            'id': 'id'
        }, inplace=True)

merged_restante = merged_restante[[col for col in s12_final.columns]]

# Consolidação final
df_final = pd.concat([s12_final, merged_restante], ignore_index=True)

s1010_utilizados = df_final[['id_1010']].dropna().drop_duplicates()

s1010_usado_completo = s1010[s1010['id_1010'].isin(s1010_utilizados['id_1010'])].copy()

s1010_usado_completo['periodo_1010'] = pd.to_datetime(
    s1010_usado_completo['periodo_1010'], errors='coerce'
).astype('datetime64[ns]')

df_final_ = df_final.drop_duplicates(subset=['index', 'id', 'codigo_rubrica', 'identificador_rubrica'])\
    .drop(columns=['index', 'id_1010'], errors='ignore').fillna('')


In [12]:
# 1200x1010

import numpy as np

num_linhas, num_colunas = df_final_.shape
resultado = f"🔹 Total de Linhas: {num_linhas}\n🔹 Total de Colunas: {num_colunas}\n\n📊 Detalhes das Colunas:\n"

for coluna in df_final_.columns:
    linhas_preenchidas = df_final_[coluna].count()  # Conta valores não nulos
    
    # Verifica se a coluna está na lista de palavras-chave
    if any(palavra.lower() in coluna.lower() for palavra in PALAVRAS_CHAVE):
            # Soma
            soma_valores = (
                df_final_[coluna]
                .replace('', np.nan)
                .astype(float)
                .sum(skipna=True)
            )

            # Exibição em formato brasileiro
            valor_formatado = f"{soma_valores:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")

            resultado += f"\n📌 Coluna: {coluna} (🔄 Convertida para Float e Substituição de '.' por ',')\n"
            resultado += f"   🔹 Linhas Preenchidas: {linhas_preenchidas}\n"
            resultado += f"   🔹 Total Somado: {soma_valores:,.2f}\n"
    else:
        resultado += f"\n📌 Coluna: {coluna}\n"
        resultado += f"   🔹 Linhas Preenchidas: {linhas_preenchidas}\n"

# Copiar para a área de transferência
pyperclip.copy(resultado)

# Exibir os resultados
print("\n✅ Resultado copiado para a área de transferência!")


✅ Resultado copiado para a área de transferência!
